In [7]:
# Imports
import numpy as np
import numpy.random as rnd
from numpy import linalg as LA
from collections import Counter
from numpy.linalg import matrix_power

## General

In [8]:
# Transition Probabilities
# for patroling behaviour
P1 = np.matrix([
    [0.30, 0.20, 0.50], 
    [0.50, 0.30, 0.20], 
    [0.20, 0.50, 0.30]
], dtype=np.float32)

# Transition Probabilities
# for guarding behaviour
P2 = np.matrix([
    [0.25, 0.10, 0.25], 
    [0.50, 0.80, 0.50], 
    [0.25, 0.10, 0.25]
], dtype=np.float32)

# Possible initial
# state distributions
PI = np.matrix([
    [1, 0, 0], 
    [0, 1, 0], 
    [0, 0, 1]
], dtype=np.float32)

# Time discrite time
# steps for MC
t = [1, 2, 4, 8, 16]

## 14.1 Markov Chains

### P2 (guarding)

In [9]:
# Compute state vectors
# results for P2 matrix
PI2_T = [[] for _ in range(3)]

for x in range(3):
    for i in t:
        PI2_T[x].append(np.dot(matrix_power(P2, i), PI[:, x]))

In [10]:
for x in range(3):
    print("\nStarting state: ", PI[:, x].T, end='\n\n')
    for idx, i in enumerate(t):
        print(f"Resulting state after {i} timesteps: ", PI2_T[x][idx].T, end='\n')


Starting state:  [[1. 0. 0.]]

Resulting state after 1 timesteps:  [[0.25 0.5  0.25]]
Resulting state after 2 timesteps:  [[0.175 0.65  0.175]]
Resulting state after 4 timesteps:  [[0.14575    0.70849997 0.14575   ]]
Resulting state after 8 timesteps:  [[0.14288059 0.7142388  0.14288059]]
Resulting state after 16 timesteps:  [[0.14285715 0.7142857  0.14285715]]

Starting state:  [[0. 1. 0.]]

Resulting state after 1 timesteps:  [[0.1 0.8 0.1]]
Resulting state after 2 timesteps:  [[0.13000001 0.74       0.13000001]]
Resulting state after 4 timesteps:  [[0.14170001 0.7166     0.14170001]]
Resulting state after 8 timesteps:  [[0.14284778 0.71430445 0.14284778]]
Resulting state after 16 timesteps:  [[0.14285715 0.7142857  0.14285715]]

Starting state:  [[0. 0. 1.]]

Resulting state after 1 timesteps:  [[0.25 0.5  0.25]]
Resulting state after 2 timesteps:  [[0.175 0.65  0.175]]
Resulting state after 4 timesteps:  [[0.14575    0.70849997 0.14575   ]]
Resulting state after 8 timesteps:  [[0.

In [26]:
# Eigen decomposition for P2
w2, v2 = LA.eig(P2)

# State is stationary
print("Process converges: ", (v2[:, 0] / np.sum(v2[:, 0])).T)

Process converges:  [[0.14285715 0.7142858  0.14285715]]


### P1 (patroling)

In [12]:
# Compute state vectors
# results for P1 matrix
PI1_T = [[] for _ in range(3)]

for x in range(3):
    for i in t:
        PI1_T[x].append(np.dot(matrix_power(P1, i), PI[:, x]))

In [13]:
for x in range(3):
    print("\nStarting state: ", PI[:, x].T, end='\n\n')
    for idx, i in enumerate(t):
        print(f"Resulting state after {i} timesteps: ", PI1_T[x][idx].T, end='\n')


Starting state:  [[1. 0. 0.]]

Resulting state after 1 timesteps:  [[0.3 0.5 0.2]]
Resulting state after 2 timesteps:  [[0.29 0.34 0.37]]
Resulting state after 4 timesteps:  [[0.3357     0.3341     0.33020002]]
Resulting state after 8 timesteps:  [[0.33333418 0.3333468  0.33331913]]
Resulting state after 16 timesteps:  [[0.33333343 0.33333343 0.33333343]]

Starting state:  [[0. 1. 0.]]

Resulting state after 1 timesteps:  [[0.2 0.3 0.5]]
Resulting state after 2 timesteps:  [[0.37       0.29000002 0.34000003]]
Resulting state after 4 timesteps:  [[0.33020005 0.33570004 0.33410004]]
Resulting state after 8 timesteps:  [[0.33331916 0.3333342  0.33334684]]
Resulting state after 16 timesteps:  [[0.33333346 0.33333346 0.33333346]]

Starting state:  [[0. 0. 1.]]

Resulting state after 1 timesteps:  [[0.5 0.2 0.3]]
Resulting state after 2 timesteps:  [[0.34000003 0.37       0.29000002]]
Resulting state after 4 timesteps:  [[0.33410004 0.33020002 0.33570004]]
Resulting state after 8 timesteps:

In [27]:
# Eigen decomposition for P1
w1, v1 = LA.eig(P1)

# State is stationary
print("Process converges: ", (v1[:, 0] / np.sum(v1[:, 0])).T)

Process converges:  [[0.3333333+0.j 0.3333333+0.j 0.3333333+0.j]]


## 14.2 Markov Process (Sampling)

In [28]:
# Generate sequences based
# on a given transtion matrix
states = ['A', 'B', 'C']
indices = range(len(states))
state2index = dict(zip(states, indices))
index2state = dict(zip(indices, states))

def generateStateSequence(X0, P, tau):
    sseq = [X0]
    iold = state2index[X0]
    
    for t in range(tau):
        inew = rnd.choice(indices, p=np.asarray(P2[:,iold]).flatten())
        sseq.append(index2state[inew])
        iold = inew
        
    return sseq

In [29]:
# Sequences
sequences = []

# Counter for analysis
counter = Counter()

# Generate sequences
for _ in range(10000):
    sequences.append(generateStateSequence('B', P2, 9))

In [31]:
# Observe last sequence for all sequences
for sequence in sequences:
    counter[sequence[-1]] += 1

In [18]:
counter

Counter({'B': 7174, 'C': 1438, 'A': 1388})

**From the counter we notice that the most likely element for the generated sequences is B.**

In [19]:
# Compute probability of B
probA = counter["A"] / sum(counter.values())
probB = counter["B"] / sum(counter.values())
probC = counter["C"] / sum(counter.values())

In [20]:
print("Probability of A: ", probA)
print("Probability of B: ", probB)
print("Probability of C: ", probC)

Probability of A:  0.1388
Probability of B:  0.7174
Probability of C:  0.1438
